# Лабораторная работа №4. Оптимизация входных параметров

## Цель оптимизации
Увеличить количество домиков, поступающих к продаже

## Алгоритм
Используется полный перебор ключевых значений, влияющих на целевую функцию: `PLANNED_HOUSES_NUM`, `CATS_NUM`, `BUILDERS_NUM`, `wooden_processing_time`, `fabric_processing_time`
Для ускорения оптимизации и удобства параметры ``wooden_processing_time`, `fabric_processing_time` сгруппированы в один `DETAIL_PROCESSING_TIME_OVERRIDE`

In [6]:
import simpy
from pprint import pprint
from itertools import product
from cathousefactory import CatHouseFactory, CatFactoryConfig
from customrng import CustomRNG
from statsprocessing import extract_business_metrics

In [7]:
def run_simulation(config):
    
    rng = CustomRNG(12345)
    env = simpy.Environment()
    factory = CatHouseFactory(env, config, rng)
    
    for i in range(1, 100):
        factory.run(until=i*100000)
    
    return factory.get_stats()

In [8]:
PARAM_VARIAIONS = {
    "PLANNED_HOUSES_NUM": [50, 150, 250],
    "CATS_NUM": [2, 6, 10],
    "BUILDERS_NUM": [2, 6, 10],
    "DETAIL_PROCESSING_TIME_OVERRIDE": [0.2, 1.0, 2.0],
}

In [9]:
# на всякий случай
PARAM_ORDER = [
    "PLANNED_HOUSES_NUM", 
    "CATS_NUM", 
    "BUILDERS_NUM", 
    "DETAIL_PROCESSING_TIME_OVERRIDE",
    ]

# проход по всем комбинациям
results = []
all_combinations = list(product(*(PARAM_VARIAIONS[key] for key in PARAM_ORDER)))
for i, values in enumerate(all_combinations):
    param_values = dict(zip(PARAM_ORDER, values))
    config = CatFactoryConfig(**param_values)
    stats = run_simulation(config)
    result = extract_business_metrics(stats)
    results.append((param_values, result))
    if i % 10 == 0:
        print(f"progress: {i}/{len(all_combinations)}")


progress: 0/81
progress: 10/81
progress: 20/81
progress: 30/81
progress: 40/81
progress: 50/81
progress: 60/81
progress: 70/81
progress: 80/81


In [11]:
business_metrics = ['houses_per_time', 'house_success_rate', 'cat_approval_rate']

for metric in business_metrics:
    best_result = max(results, key=lambda res: res[1][metric])
    print("#################################################################")
    print(f"Лучший результат для метрики {metric}: {best_result[1][metric]}")
    print(f"Достигается при следующих параметрах:")
    pprint(best_result[0])
    print(f"Полные значения метрик для таких параметров:")
    pprint(best_result[1])
    print()

#################################################################
Лучший результат для метрики houses_per_time: 0.09239547282761201
Достигается при следующих параметрах:
{'BUILDERS_NUM': 10,
 'CATS_NUM': 10,
 'DETAIL_PROCESSING_TIME_OVERRIDE': 0.2,
 'PLANNED_HOUSES_NUM': 250}
Полные значения метрик для таких параметров:
{'cat_approval_rate': 0.6394355256343863,
 'house_success_rate': 0.5986666666666667,
 'houses_per_time': 0.09239547282761201}

#################################################################
Лучший результат для метрики house_success_rate: 0.5994612794612795
Достигается при следующих параметрах:
{'BUILDERS_NUM': 10,
 'CATS_NUM': 6,
 'DETAIL_PROCESSING_TIME_OVERRIDE': 0.2,
 'PLANNED_HOUSES_NUM': 150}
Полные значения метрик для таких параметров:
{'cat_approval_rate': 0.6420946335833814,
 'house_success_rate': 0.5994612794612795,
 'houses_per_time': 0.06402197833810393}

#################################################################
Лучший результат для метрики cat_ap

## Анализ результатов:

Наилучший ключевые показатель `houses_per_time` достигается при минимальных параметрах wooden_processing_time и fabric_processing_time, а также при максимальных BUILDERS_NUM и CATS_NUM

Дополнительные показатели `cat_approval_rate` и `house_success_rate` не особо отличаются от своих средних значений

## Рекомендации
 
Увеличить количество обрабатывающих ресурсов - больше станков, чтобы уменьшить `wooden/fabric_processing_time`, а также больше сборщиков и котов 